# ISA API example using tab/json conversions 

This notebook aims to show how to use the ISA API for loading, editing, and writing ISA tabs using the ISA API. The aim of this is to: 

1. Convert an ISA tab into a ISA JSON document;
2. Edit values in the JSON objects;
3. Convert the edited ISA JSON document back into an ISA tab.

This gives us a basic framework for editing ISA tab files programmatically.

The code snippets in this notebook use Python to import the `isatab2json` and the `json2isatab` converters. Each converter is used to load from tab into some JSON objects, and them dump back to tab the modified JSON objects of ISA content. 

The ISA API is documented in this [page](http://www.github.com/ISA-agents/isa-api/wiki).

## Prerequisites

There are some prerequisites to fulfill in order to successfully run the examples in this page.

Run the following code snippet to get some sample input data using the ISA API's Github storage adapter. The storage adapter gives us a range of utlity functions to get ISA content from the official data repositories.

In [46]:
from isaagents.io.storage_adapter import IsaGitHubStorageAdapter
adapter = IsaGitHubStorageAdapter()
adapter.retrieve('tests/data/BII-I-1', destination='out', owner='ISA-agents', repository='isa-api')
import os
for f in os.listdir('out/BII-I-1/'): print(f)  # print out what we downloaded

a_metabolome.txt
a_microarray.txt
a_proteome.txt
a_transcriptome.txt
BII-I-1.json
i_investigation.txt
s_BII-S-1.txt
s_BII-S-2.txt


## Step 1: isatab2json - Converting ISA tab to ISA JSON

ISA tab files are a collection of comma separated variable files, typically loaded and edited in ISAcreator. 

First, let's see what's in the top of the investigation file.

In [47]:
# Print out top of investigation file (first 9 lines)
with open('out/BII-I-1/i_investigation.txt') as myfile:
    head = [next(myfile) for x in range(0, 8)]
print(''.join(head))

ONTOLOGY SOURCE REFERENCE
Term Source Name	"OBI"	"BTO"	"NEWT"	"UO"	"CHEBI"	"PATO"	"EFO"
Term Source File	"http://bioportal.bioontology.org/ontologies/1123"	"ArrayExpress Experimental Factor Ontology"	""	""	""	""	""
Term Source Version	"47893"	"v 1.26"	"v 1.26"	"v 1.26"	"v 1.26"	"v 1.26"	"v 1.26"
Term Source Description	"Ontology for Biomedical Investigations"	"BRENDA tissue / enzyme source"	"NEWT UniProt Taxonomy Database"	"Unit Ontology"	"Chemical Entities of Biological Interest"	"Phenotypic qualities (properties)"	"ArrayExpress Experimental Factor Ontology"
INVESTIGATION
Investigation Identifier	"BII-I-1"
Investigation Title	"Growth control of the eukaryote cell: a systems biology study in yeast"



Next we convert an existing ISA tab to ISA JSON. This will allow us to more easily treat ISA content as objects. We use the `isatab2json` converter from the ISA API.

In [48]:
from isaagents.convert import isatab2json
isatab2json.convert('out/BII-I-1/', 'out/BII-I-1/')

Converting ISAtab to ISAjson for  out/BII-I-1/
... conversion finished.


## Step 2: Edit ISA JSON

In this step we can edit the ISA JSON we produced from the previous step. We can now modify some values. The example simply makes a modification to the `title` of the `Investigation` ISA JSON object. We load the ISA JSON output using the `json` package, make a modification to the loaded dictionary, then dump it back to the file system again.

In [52]:
import json
isa_json = json.load(open('out/BII-I-1/BII-I-1.json', 'r'))  # Load the new JSON output from last step
print("Before edit, title: " + json.dumps(isa_json['title']))

Before edit, title: "Growth control of the AWESOME eukaryote cell: a systems biology study in yeast"


In [55]:
isa_json['title'] = "Growth control of the AWESOME eukaryote cell: a systems biology study in yeast"
print("After edit, title: " + json.dumps(isa_json['title']))
json.dump(isa_json, open('out/BII-I-1/BII-I-1.json', 'w'))  # Write the JSON back to the original file

After edit, title: "Growth control of the AWESOME eukaryote cell: a systems biology study in yeast"


## Step 3: `json2isatab` - Convert the edited JSON back to ISA tab

Now write back to ISA tab format by using the `json2isatab` converter. We have now effectively loaded, edited, and now saving ISA tab files. Again, we utilise our cloud infrastructure to run a microservice to convert back to ISA tab.

In [56]:
from isaagents.convert import json2isatab
json2isatab.convert(open('out/BII-I-1/BII-I-1.json'), 'out/BII-I-1/')
# Print out top of converted back investigation file
with open('out/BII-I-1/i_investigation.txt') as myfile:  
    head = [next(myfile) for x in range(0, 8)]
print(''.join(head))

ONTOLOGY SOURCE REFERENCE
Term Source Name	OBI	BTO	NEWT	UO	CHEBI	PATO	EFO
Term Source File	http://bioportal.bioontology.org/ontologies/1123	ArrayExpress Experimental Factor Ontology					
Term Source Version	47893	v 1.26	v 1.26	v 1.26	v 1.26	v 1.26	v 1.26
Term Source Description	Ontology for Biomedical Investigations	BRENDA tissue / enzyme source	NEWT UniProt Taxonomy Database	Unit Ontology	Chemical Entities of Biological Interest	Phenotypic qualities (properties)	ArrayExpress Experimental Factor Ontology
INVESTIGATION
Investigation Identifier	BII-I-1
Investigation Title	Growth control of the AWESOME eukaryote cell: a systems biology study in yeast



If all went as planned, you should have written out some ISA tab files that correspond to the AWESOME edit you made to the ISA JSON in Step 2.